<a href="https://colab.research.google.com/github/rinohamanishi/Visualizing-Seattle/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Group 2 Lab 1 Welcome to Seattle

**Theme**: The flow of domestic flights coming into Seattle in one month

**Data source**: [Bureau of Transportation Statistics](https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGJQO_fu146_anzr=b0-gvzr)   
**Data table**: [lab1.csv ](https://drive.google.com/file/d/1FMWyDVos8BRMC2RA-IOUlTsRsUCYWjP9/view?usp=sharing)  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_flight_num = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lab1.csv')

df_flight_num

,January 2023 U.S. Flight
ORIGIN_CITY_NAME,DEST_CITY_NAME
"Allentown/Bethlehem/Easton, PA","Atlanta, GA"
"Allentown/Bethlehem/Easton, PA","Atlanta, GA"
"Allentown/Bethlehem/Easton, PA","Atlanta, GA"
"Allentown/Bethlehem/Easton, PA","Atlanta, GA"
...,...
"Yuma, AZ","Phoenix, AZ"
"Yuma, AZ","Phoenix, AZ"
"Yuma, AZ","Phoenix, AZ"
"Yuma, AZ","Phoenix, AZ"


In [ ]:
# @title Default title text
import pandas as pd

# Load the data from the CSV file
file_path = '/content/drive/MyDrive/Colab Notebooks/lab1.csv'
df_flight_num = pd.read_csv(file_path, header=1)

# Grouping by origin and destination and counting the number of flights
flight_counts = df_flight_num.groupby(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME']).size().reset_index(name='FLIGHT_COUNT')

# Output the result
print(flight_counts)


     ORIGIN_CITY_NAME         DEST_CITY_NAME  FLIGHT_COUNT
0        Aberdeen, SD        Minneapolis, MN            61
1         Abilene, TX  Dallas/Fort Worth, TX           107
2     Adak Island, AK          Anchorage, AK             8
3       Aguadilla, PR    Fort Lauderdale, FL            45
4       Aguadilla, PR           New York, NY            62
...               ...                    ...           ...
5150     Wrangell, AK         Petersburg, AK            30
5151      Yakutat, AK            Cordova, AK            30
5152      Yakutat, AK             Juneau, AK            30
5153         Yuma, AZ  Dallas/Fort Worth, TX            31
5154         Yuma, AZ            Phoenix, AZ            59

[5155 rows x 3 columns]


In [ ]:
# @title
import pandas as pd

# Load the data from the CSV file
file_path = '/content/drive/MyDrive/Colab Notebooks/lab1.csv'
df_flight_num = pd.read_csv(file_path, header=1)

# Filter for flights where the destination is Seattle and get the top 5 origins
seattle_flights = df_flight_num[df_flight_num['DEST_CITY_NAME'] == 'Seattle, WA']
top_5_seattle_origins = seattle_flights.groupby(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME']).size().reset_index(name='FLIGHT_COUNT')
top_5_seattle_origins = top_5_seattle_origins.sort_values(by='FLIGHT_COUNT', ascending=False).head(5)

# For each of these top 5 origins, find the top origin going to them
top_flights = pd.DataFrame()
for city in top_5_seattle_origins['ORIGIN_CITY_NAME']:
    city_flights = df_flight_num[df_flight_num['DEST_CITY_NAME'] == city]
    top_city_flights = city_flights.groupby(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME']).size().reset_index(name='FLIGHT_COUNT')
    top_city_flight = top_city_flights.sort_values(by='FLIGHT_COUNT', ascending=False).head(2)
    top_flights = pd.concat([top_flights, top_city_flight], ignore_index=True)

# Combine the two sets of data
final_data = pd.concat([top_5_seattle_origins, top_flights], ignore_index=True)

# print(final_data)

import plotly.graph_objects as go

# Create lists of unique origin and destination cities
unique_cities = pd.concat([final_data['ORIGIN_CITY_NAME'], final_data['DEST_CITY_NAME']]).unique()
#print(unique_cities)

# Map each city to a unique index
city_to_index = {city: idx for idx, city in enumerate(unique_cities)}

# Create source, target, and value lists for the Sankey diagram
sources = final_data['ORIGIN_CITY_NAME'].map(city_to_index)
targets = final_data['DEST_CITY_NAME'].map(city_to_index)
values = final_data['FLIGHT_COUNT']

"""
 Generate a unique color for each link in the diagram
 import matplotlib.colors as mcolors
 all_colors = list(mcolors.CSS4_COLORS.values())
 link_colors = all_colors[:len(sources)]

# Define node colors
node_colors = ["#e74c3c", "#e91e63", "#AED6F1", "#85929E", "#2E4053",
               "#8E44AD", "#E67E22", "#BA4A00", "#E74C3C", "#28B463",
               "#F5B041", "#F7DC6F", "#F7DC6F", "#F1948A", "#F7DC6F",
               "#F5B041", "#F7DC6F", "#F7DC6F"]
if len(node_colors) < len(unique_cities):
    node_colors *= (len(unique_cities) // len(node_colors) + 1)
"""
# Dictionary mapping states to colors
state_colors = {
    'AZ': "#381D2A",  # Dark purple
    'NV': "#3B435D",  # Delft blue
    'CA': "#AABD8C",  # Olivine
    'AK': "#3E6990",  # Lapis Lazuli
    'CO': "#E9E3B4",  # Vanilla
    'IL': "#F39B6D",  # Atomic tangerine
    'NY': "#EEBF91",  # Peach
    'WA': "#74938E"   # Cambridge blue
}


# Define the list of colors
final_data['NODE_COLOR'] = final_data['ORIGIN_CITY_NAME'].str[-2:].map(state_colors)
#print(final_data)
node_colors = final_data['NODE_COLOR'].tolist()
#print(node_colors)

# Create the Sankey diagram with custom colors for nodes and links
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_cities,
        color=node_colors  # Use the defined colors for nodes
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=node_colors  # Use the defined colors for each link
    )
)])

unique_city_colors = final_data[['ORIGIN_CITY_NAME', 'NODE_COLOR']].drop_duplicates()
city_color_dict = dict(zip(unique_city_colors['ORIGIN_CITY_NAME'], unique_city_colors['NODE_COLOR']))

# Add legend as annotations for cities
for i, (city, color) in enumerate(city_color_dict.items()):
    fig.add_annotation(
        x=1.1,
        y=1 - (i * 0.1),
        xref="paper",
        yref="paper",
        text=city,  # City name
        showarrow=False,
        font=dict(size=10, color="white"),
        align="left",
        width=110,
        height=20,
        bgcolor=color,
        bordercolor="white",
        borderwidth=1
    )

fig.add_annotation(x=0,y=1.1,
                   text="Rino H, Bella L, Confidence O, Mars O",
                   showarrow=False)
fig.add_annotation(x=0,y=1.05,
                   text="January 22 2024",
                   showarrow=False)
fig.add_annotation(x=0,y=1,
                   text="Source: Bureau of Transportation",
                   showarrow=False)

fig.update_layout(title_text="Flights into Seattle, WA in January 2023", font_size=10)
fig.show()

     ORIGIN_CITY_NAME     DEST_CITY_NAME  FLIGHT_COUNT
0         Phoenix, AZ        Seattle, WA           601
1       Las Vegas, NV        Seattle, WA           600
2     Los Angeles, CA        Seattle, WA           574
3   San Francisco, CA        Seattle, WA           535
4       Anchorage, AK        Seattle, WA           521
5          Denver, CO        Phoenix, AZ           764
6         Chicago, IL        Phoenix, AZ           663
7     Los Angeles, CA      Las Vegas, NV           965
8          Denver, CO      Las Vegas, NV           753
9       Las Vegas, NV    Los Angeles, CA           962
10  San Francisco, CA    Los Angeles, CA           886
11    Los Angeles, CA  San Francisco, CA           900
12       New York, NY  San Francisco, CA           556
13        Seattle, WA      Anchorage, AK           523
14         Juneau, AK      Anchorage, AK            91
